###### Name: Abd-el-rahman Zaglool

###### ID: 5107746

###### Project: News classification

For the project, I'll collect the news from The New York Times webpage. Articles will be retrieved from 8 different sections:
1. Business
2. Science
3. Health
4. Sports
5. Arts
6. Style 
7. Food
8. Travel

In [9]:
# importing the necessary packages

import requests
import time
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

The New York Times offer a wide range of handy APIs. One of them is 'Times Wire API' that allows to retrieve the real-time feed of NYT published articles.

To fetch articles from the 'Times Wire API' I'll create a helper function named scrape_articles(). 

The function will take as inputs the desired news' section and the number of articles to retrieve ('limit' parameter).
 
The function returns the response as a JSON object.

In [10]:
def scrape_articles(section, limit):
  requestUrl = requestUrl = 'https://api.nytimes.com/svc/news/v3/content/all/'+section+'.json?limit='+str(limit)+'&api-key=X5IOvJkmFhSRsE7jrZHfMM25hEATDJpS'
  requestHeaders = {
    "Accept": "application/json"
  }

  response = requests.get(requestUrl, headers=requestHeaders).json()

  return response

I'll also define a second helper function in order to extract the necessary fields ('title', 'abstract' and 'section' of the article) from the response.

In [11]:
def retrieve_articles(response):
        articles = []
        docs = response['results']
        for doc in docs:
                filteredDoc = {}
                filteredDoc['title'] = doc['title']
                filteredDoc['abstract'] = doc['abstract']
                filteredDoc['section'] = doc['section']
                articles.append(filteredDoc)

        return articles        

Now I'll use the two functions defined above to iterate articles retrieval and fields extraction over all of the 8 sections.

The result will be a list of lists (8) where each list contains articles from one section only. 

The articles are stored as dictionaries with three keys: 'title', 'abstract' and 'section'.

From each section I've collected at most 500 articles. Number of articles collected may change among sections depending on the feed at the time of code execution.


In [12]:
sections = ['business', 'science', 'health', 'sports', 'arts', 'style', 'food', 'travel']
limit = 500
news = []

for section in sections:

    response = scrape_articles(section, limit)
    section_articles = retrieve_articles(response)
    news.append(section_articles)
    time.sleep(12)    

# time elapsed 1m 57s    

Now I'll store the retrieved news in a DataFrame

In [13]:
# Create an empty DataFrame
df = pd.DataFrame()

# Iterate over each list of dictionaries
for sublist in news:
    # Create a temporary DataFrame for each list of dictionaries
    temp_df = pd.DataFrame(sublist)
    
    # Concatenate the temporary DataFrame with the main DataFrame
    df = pd.concat([df, temp_df], ignore_index=True) 

In [14]:
df

,title,abstract,section
0,The Ukraine War Changed This Company Forever,Nokian Tyres of Finland made 80 percent of its...,Business
1,"In Business, ‘Flat’ Structures Rarely Work. Is...",Companies that have rejected hierarchies have ...,Business
2,Federal Judge Limits Biden Officials’ Contacts...,The order came in a lawsuit filed by the attor...,Business
3,Meta Loses Appeal on How It Harvests Data in G...,The ruling by the European Court of Justice cl...,Business
4,Corporate Landscaping Lets Its Hair Down,More companies are eschewing manicured grass i...,Business
...,...,...,...
2991,A Caribbean Island’s Audacious Tourism Experiment,"To step onto Montserrat’s volcanic beaches, 21...",Travel
2992,"Quarantine and Travel: Strict Penalties, Rare ...",As quarantine requirements for travelers have ...,Travel
2993,"Traveling the World, While Looking Over Her Sh...",The number of women traveling the world solo i...,Travel
2994,Luxury Comes to Expedition Cruising,New small ships built for adventure are also o...,Travel


In [16]:
# visualizing number of articles retrieved by section
df.section.value_counts()

section
Business    500
Sports      500
Arts        500
Style       500
Food        375
Science     299
Health      175
Travel      147
Name: count, dtype: int64

In [15]:
# dropping rows with NaN on both title and abstract:
df = df.drop(index= df[(df['title'].isna()) & df['abstract'].isna()].index)
df.reset_index(drop=True, inplace=True)

In [21]:
# Concatenating title and abstract into the same 'article' column
# Some rows have NA in title, others have NA in abstract so I have to handle the different cases 

df.loc[df[df['abstract'].isna()].index, 'title'] = df[df['abstract'].isna()]['title'].values #NA in abstract 
df.loc[df[(df['title'].notnull()) & (df['abstract'].notnull())].index, 'title'] = df[(df['title'].notnull()) & (df['abstract'].notnull())]['title'].values + ' : ' + df[(df['title'].notnull()) & (df['abstract'].notnull())]['abstract'].values #no NAs
df.loc[df[df['title'].isna()].index, 'title'] = df[df['title'].isna()]['abstract'].values #NA in title
df.rename(columns={'title' : 'article'}, inplace=True)
df.drop(columns= df.columns[1], axis=1, inplace=True)
df    

,article,section
0,The Ukraine War Changed This Company Forever :...,Business
1,"In Business, ‘Flat’ Structures Rarely Work. Is...",Business
2,Federal Judge Limits Biden Officials’ Contacts...,Business
3,Meta Loses Appeal on How It Harvests Data in G...,Business
4,Corporate Landscaping Lets Its Hair Down : Mor...,Business
...,...,...
2991,A Caribbean Island’s Audacious Tourism Experim...,Travel
2992,"Quarantine and Travel: Strict Penalties, Rare ...",Travel
2993,"Traveling the World, While Looking Over Her Sh...",Travel
2994,Luxury Comes to Expedition Cruising : New smal...,Travel


Now I'll set up the necessary components for text preprocessing tasks. 

Preprocessing involve text tokenization, stop words removal, part-of-speech mapping and lemmatization.

In [22]:
stop_words = set(stopwords.words('english')) #retrieving english stop words list

tokenizer = RegexpTokenizer(r'\w+') #initializing tokenizer

def get_wordnet_pos(word):
    """Maps the input word's POS tag to the appropriate WordNet POS tag"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer() #initializing lemmatizer

In [24]:
# Implementing a for loop to preprocess each article of the news dataframe

nrows= df.shape[0]
corpus = []

for i in range(0, nrows):
    globals()['article%s' % i] = df.iloc[i,0]

    globals()['word_tokens%s' % i] = tokenizer.tokenize(globals()['article%s' % i]) #tokenization by word
    globals()['filtered_sentence%s' % i] = [w.lower() for w in globals()['word_tokens%s' % i] if not w.lower() in stop_words]  # insert only words that aren't present in stop_words. In lowercase

    globals()['filtered_sentence_lemmatized%s' % i] = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in globals()['filtered_sentence%s' % i]] #lemmatize the worlds in globals()['filtered_sentence%s' % i]
    globals()['filtered_sentence_lemmatized%s' % i] = ' '.join(globals()['filtered_sentence_lemmatized%s' % i])
    corpus.append(globals()['filtered_sentence_lemmatized%s' % i])

# time elapsed: 5m 14s

I'll save then the processed articles as a new column in my DataFrame.

In addition, I'll factorize the sections (i.e. assign a different number to each section). The factorized_section column will be my target variable in the news classification problem.  

In [25]:
df['processed_article'] = corpus
df['factorized_section'] = df['section'].factorize()[0]
df.dropna(inplace=True)
df

,article,section,processed_article,factorized_section
0,The Ukraine War Changed This Company Forever :...,Business,ukraine war change company forever nokian tyre...,0
1,"In Business, ‘Flat’ Structures Rarely Work. Is...",Business,business flat structure rarely work solution c...,0
2,Federal Judge Limits Biden Officials’ Contacts...,Business,federal judge limit biden official contact soc...,0
3,Meta Loses Appeal on How It Harvests Data in G...,Business,meta loses appeal harvest data germany ruling ...,0
4,Corporate Landscaping Lets Its Hair Down : Mor...,Business,corporate landscape let hair company eschew ma...,0
...,...,...,...,...
2991,A Caribbean Island’s Audacious Tourism Experim...,Travel,caribbean island audacious tourism experiment ...,7
2992,"Quarantine and Travel: Strict Penalties, Rare ...",Travel,quarantine travel strict penalty rare enforcem...,7
2993,"Traveling the World, While Looking Over Her Sh...",Travel,travel world look shoulder number woman travel...,7
2994,Luxury Comes to Expedition Cruising : New smal...,Travel,luxury come expedition cruise new small ship b...,7


In [26]:
# Show the factorization legend
df[['section', 'factorized_section']].drop_duplicates().sort_values('factorized_section')

,section,factorized_section
0,Business,0
500,Science,1
799,Health,2
974,Sports,3
1474,Arts,4
1974,Style,5
2474,Food,6
2849,Travel,7


To be fed to the classification model, the processed articles must be converted into a numerical representation. In particular, with  the help Scikit-learn's CountVectorizer class I'll represent the corpus as a sparse matrix with n rows = n processed articles and n columns = n unique words in the corpus. 

In [27]:
x = df.iloc[:,2].values
y = df.factorized_section.values

# Init CountVectorizer
cv = CountVectorizer()
# Fit to the corpus
x = cv.fit_transform(x).toarray()

Now I only need to split the data in train (80% of data) and test (the remaining 20%) portions, and I'll be ready to call the classification model

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42, shuffle = True)

To classify the articles I chose a basic Logistic Regression since it offers a good trade-off between performance and complexity (i.e. computational expense). Given that news classification problem involves multiple labels, I'll resort to OneVsRestClassifier() class. 

OneVsRest takes a binary classifier (the Logistic Regression) and wraps it to handle multiclass classification problems by decomposing them into multiple binary classification tasks.

To evaluate the predictive power of the model, considering the equal importance of all labels, I relied on the accuracy score.

In [29]:
mdl = LogisticRegression(random_state=42)

oneVsRest = OneVsRestClassifier(mdl)

oneVsRest.fit(x_train, y_train)

y_pred = oneVsRest.predict(x_test)

# Performance
accuracy = round(accuracy_score(y_test, y_pred)*100, 2)
print(f'Accuracy Score of Basic Logistic Regression: {accuracy} %')

Accuracy Score of Basic Logistic Regression: 77.5 %


The Accuracy score measures the proportion of correctly predicted labels compared to the total number of samples. On average, our Logistic regression correctly predicts the section of 8 articles out of 10! That's already a great result, which can be further enhanced by, for example, employing some hyperparameter fine-tuning...